In [1]:
import numpy as np
import pandas as pd
from statsmodels import stats
from statsmodels.stats.proportion import proportion_confint
import scipy

In [2]:
def proportions_diff_confint_ind(data, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    n1 = data.iloc[0]['sum']
    n2 = data.iloc[1]['sum']
    
    p1 = float(data.iloc[0]['pos']) / n1
    p2 = float(data.iloc[1]['pos']) / n2
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(data):
    n1 = data.iloc[0]['sum']
    n2 = data.iloc[1]['sum']
    
    p1 = float(data.iloc[0]['pos']) / n1
    p2 = float(data.iloc[1]['pos']) / n2
    
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [3]:
# Данные задачи
n = 1000000
clicks1 = 10000
clicks2 = 11500
inst1 = 440
inst2 = 500

In [4]:
# Проверка гипотезы о том, что доля установок к кликам в двух экспериментах одинакова
data = np.array([[inst1, clicks1 - inst1, clicks1], [inst2, clicks2 - inst2, clicks2]])
data = pd.DataFrame(data)
data.columns = ['pos', 'neg', 'sum']
print(data)
print(proportions_diff_confint_ind(data))
print(proportions_diff_z_test(proportions_diff_z_stat_ind(data)))

   pos    neg    sum
0  440   9560  10000
1  500  11000  11500
(-0.004960115555974521, 0.0060035938168440835)
0.8519626637290214


Гипотезу нельзя отвергнуть.

In [5]:
# Проверка гипотезы о том, что доля кликов в двух экспериментах одинакова
data = np.array([[clicks1, n - clicks1, n], [clicks2, n - clicks2, n]])
data = pd.DataFrame(data)
data.columns = ['pos', 'neg', 'sum']
print(data)
print(proportions_diff_confint_ind(data))
print(proportions_diff_z_test(proportions_diff_z_stat_ind(data)))

     pos     neg      sum
0  10000  990000  1000000
1  11500  988500  1000000
(-0.001785830694352124, -0.0012141693056478753)
0.0


Можем отвергнуть гипотезу о том, что кликабельность двух баннеров одинаковая. По доверительному интервалу можно судить о том, что доля кликов по второму баннеру выше, чем по первому.

In [23]:
# Проверка гипотезы о том, что доля установок в двух экспериментах одинакова
data = np.array([[inst1, n - inst1, n], [inst2, n - inst2, n]])
data = pd.DataFrame(data)
data.columns = ['pos', 'neg', 'sum']
print(data)
# data = np.array([[440, 500], [10000-440, 11500 - 500], [10000, 11500]])
print(proportions_diff_confint_ind(data))
print(proportions_diff_z_test(proportions_diff_z_stat_ind(data)))

   pos     neg      sum
0  440  999560  1000000
1  500  999500  1000000
(-0.00012007717719999515, 7.717719999516192e-08)
0.05029525763665932


Не можем отвергнуть гипотезу о том, что количество установок по второму баннеру отличается от первого баннера на достигаемом уровне значимости 0.05. Тем не менее, p-value достаточно близко к 0.05.

# Выводы

Имеем предположительно независимые выборки, так как пользователи выбирались случайно.

Можем выдвинуть гипотезу о том, что доля кликов по первому и второму баннеру одинакова. Такая гипотеза отвергается на достигаемом уровне значимости 0.05 при двусторонней альтернативе. По 95% доверительному интервалу разности долей можем утверждать, что доля кликов по второму баннеру превышает долю кликов по первому. Если говорить о долях установок, то не можем отвергнуть гипотезу об их равенстве на уровне значимости 0.05, но можем отвергнуть на уровне значимости 0.1, а построенный для разности долей 95% доверительный интервал лишь незначительно захватывает 0. В добавок была проверена гипотеза о равенстве долей установок к кликам для двух баннеров, которая не была отвергнута, что позволяет с большей уверенностью ориентироваться на разность в долях кликов, как показатель эффективности. В итоге можно заключить, что второй баннер является более эффективным. Тем не менее, для проверки можно продолжать показывать старый баннер некоторому небольшому проценту пользователей, что позволит продолжить оценку эффективности нового баннера.

Вообще все эти выводы справедливы при учёте того, что выборки независимы. Нужно проверить, не попали ли одни и те же пользователи в группы, тестирующие первый и второй баннеры. Такие данные могут повлиять на результат. Поэтому перед принятием окончательного решения, следует уточнить данные.